<a href="https://colab.research.google.com/github/bradshimmin/llm_experiments/blob/main/AI_Skills_TikTok_team_categories_5sep24.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Set up software
#
!pip -q install openai tiktoken


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.2 MB/s eta 0:00:00


In [2]:
# Import libraries
#
import os
import openai
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import textwrap
import tiktoken
import typing

from enum import Enum
from typing import Union
from openai import OpenAI
from pydantic import BaseModel, Field
from typing import List, Dict

print("all done")

all done


In [3]:
# Load openai key
#
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get('OpenAI_key')

---
# Set up AI Skills data

In [4]:
# Mount Google Drive
#
from google.colab import drive
drive.mount('/content/gdrive')

print("all done")

Mounted at /content/gdrive
all done


In [5]:
# Load our sample dataset using 2q24 cleaned df
#
# df_sample = pd.read_pickle("/content/gdrive/MyDrive/Colab Notebooks/Datasets/ai_skills_tracker/ai_skills_2q24/ai_skills_2q24_final_clean_28aug24.pkl")


In [67]:
# Load the data from our earlier run
df_sample = pd.read_pickle('/content/gdrive/MyDrive/Colab Notebooks/Datasets/df_ner_tiktok_30aug24.pkl')

In [68]:
df_sample.shape

(139, 33)

In [69]:
df_sample.head(n=2)

,RecordID,Job title,Full job description,Date collected,Date posted,Week,Year,Month,Quarter,Hiring entity,...,Main industry,Source name,Source URL,File Date,title,role,objectives,technologies,skills,summary
263036,263037,"Machine Learning Engineer, TikTok Trust and Sa...",Responsibilities TikTok is the leading destina...,None,2024-03-15,11,2024,3,2024Q1,TikTok,...,No data,Glassdoor,https://www.glassdoor.com/partner/jobListing.h...,2024-01-01,Machine Learning Engineer,Machine Learning Engineer - Trust and Safety,[objectives=['Design and develop advanced mach...,"[technologies=['Machine Learning', 'Active Lea...","[skills=['Strong coding skills in Python', 'So...",TikTok is seeking a Machine Learning Engineer ...
263037,263038,"Machine Learning Engineer, Risk Data Mining",Responsibilities TikTok is the leading destina...,None,2024-03-15,11,2024,3,2024Q1,TikTok,...,Entertainment Providers,LinkedIn,https://www.linkedin.com/jobs/view/machine-lea...,2024-01-01,Machine Learning Engineer - Business Risk Inte...,Machine Learning Engineer,[objectives=['Protect TikTok users including c...,"[technologies=['Linux', 'Hadoop', 'Hive', 'Spa...","[skills=['machine learning', 'software develop...",This position requires a strong background in ...


In [70]:
# Check date range
print(df_sample['Date posted'].min())
print(df_sample['Date posted'].max())


2024-03-02 00:00:00
2024-06-17 00:00:00


---
# Categorize job posts


In [15]:
# df_test = recent_tiktok_jobs.sample(n=10).copy()
# df_test.shape

In [18]:
# Set up the category list
category_list = """

Trust and Safety
- Trust
- Safety
- Privacy
- Data protection
- Governance
- Integrity
- Risk
- Fraud
- Abuse
- Misinformation / disinformation
- Harm
- Legal
- Moderation
- Policy

Monetisation
- E-commerce
- Shopping
- Sales
- Monetisation
- Advertising / ads
- Marketplace
- Shop
- Merchant services
- Payments
- Retail media
- Shoppable video / TV
- Customer journey
- Funnel
- Engagement
- Conversion

Customer / user experience
- Search
- Recommendation
- Personalisation
- Engagement
- Customer service
- Customer / user experience
- Chatbot
- Intelligent agent
- Fulfilment
- Discovery

Content
- Advertising / Ads
- Marketing
- Video
- Live-streaming
- Gaming / games
- Music
- Podcasts
- Augmented reality
- Virtual reality
- Chat / messaging
- Connected TV
- Smart TV
- Retail media
- Dubbing
- Subtitling
- Photos

Technology
- Platform
- Video delivery
- Content creation
- Generative AI
- AI
- Machine learning
- Computer vision
- Content creation
- Fulfilment
- Logistics
- Supply chain
- Data clean rooms
- Behavioral modelling
- R&D
"""

In [17]:
# set up the team list
team_list = """
- Trust and Safety team
- Business Risk Integrated Control (BRIC) team
- The TikTok Music Recommendation Team
- USDS-Platform and Community Integrity (PaCI) team
- Global e-commerce Data Science team
- e-commerce alliance team
- Global E-Commerce Alliance Team
- Global Product & RD team
- TikTok Ads Core team
- Global e-commerce business team
- Content E-commerce team
- Generative AI team under Monetization Technology
- Moderation & QA teams
- Policy Enforcement Team
- TikTok Ads Creative & Ecosystem team
- Search Growth E-commerce team
- Risk & Response Team
- US Tech & Prod.
- User Operations team
- Privacy & Legal teams
- Global Payment Decision Science team
- Global Payment team of U.S. Data Security department
- User Research and Ecommerce teams
- E-commerce - Recommendation and Marketing
- E-commerce - Alliance
- E-commerce - Search
- E-commerce - Search Growth
- PDPO (Privacy and Data Protection Office)
- The Marketing Strategy, Insights, and Analytics team
- Security Engineering team
- E-commerce Global Supply Chain and Logistics team
- The Data Solutions team
- The Applied AI (AAI) Team
- Monetization Integrity team
- Data Cycling Center
- Applied AI (DCC) Team
- Multimedia Arch team
- Global e-commerce business team
- Intelligent Customer Service under TikTok's global e-commerce business
- Data Science team of the Tech and Product department at TikTok USDS
- Governance and Experience
- US Data Loss Prevention Operations
- Global Payment team of U.S. Data Security department
- the Investigations and Insights team
- Video Platform team
- User Operations team
- Search Growth E-commerce
- Governance and Experience Algorithm team
- Data Science Search Team
- Recommendation algorithm team
- TikTok Search Team
- App Ads and Gaming team
- Security and Risk Control Organization
- E-Commerce Risk Control (ECRC) team
- content understanding and generation team of e-commerce algorithms
- GBS Analytics team
- Ecommerce's Governance and Experience(GNE)
- Data Cycling Center Team
- Live Platform Intelligence team
- GAI-Vision team
- ads data platform team
- Monetization GenAI team
- AML team
- Ads Data Team
- Data Platform Singapore Business Partnering (DPSG BP) team
- AI Platform team
- Eng-AI Innovation Center
- Search Ads team
- Web Ads and Open-loop E-commerce team
- AI Data Service and Operations (ADSO) team
- RecCore Dedup team
- Machine Learning systems team
- Moderation
- Policy
"""

In [56]:
# Set up a new, simplified Pydantic class
#
class Categories(BaseModel):
    """
    Represents the top three categories of the job posting.

    This model captures the most applicable categories to which this job posting belongs.
    The full list of five main categories, each with several sub categories to choose from is below.
    Please select a maximum of three main + sub categories such as 'Trust and Safety - Privacy',
    'Monetisation - Payments', etc.  Only pick a category + sub category if it figures into the
    job in a prominant manner. Return the top three categories and no more. Please only return a
    simple python list of strings.
    {category_list}
    """
    categories: list[str] # Added type annotation for categories

class Team(BaseModel):
    """
    Represents the team wherein the employee will work.

    This model capture the area, department, or division in which the employee
    will work within the company, using the following list. Don't worry about finding
    an exact match. But please assign this job to one of the teams listed below rather
    than making up a new team name. Just assign this post to the single closest match
    based on the following list.
    {team_list}
    """
    team: str # Added type annotation for teams

class JobResponse(BaseModel):
    """
    Represents the structured response for an article analysis.

    This model aggregates job categories and most relevant team in a job post
    """
    categories: list[Categories]
    team: str


In [57]:
# Inference function
#
client = OpenAI()

categories = []
team = ""

def parse_post(jobpost):
    completion = client.beta.chat.completions.parse(
    model="gpt-4o-mini",
    messages=[
            {"role": "system", "content": "You are a helpful assistant that \
            scans job posts and extracts important information such as job categories \
            and the team wherein the appliant will work. Please be thorough in your response. \
            And remember, this is an analysis, so be concise, and above all else, \
            do not hallucinate."},
            {"role": "user", "content": jobpost},
        ],
        response_format=JobResponse,
    )

    message = completion.choices[0].message
    if message.parsed:
        # Separate lists for technologies and business skills
        team = str(message.parsed.team)
        categories = [str(obj) for obj in message.parsed.categories]

        # Return as separate values
        return team, categories
    else:
        return "", []  # Return empty lists in case of refusal or no parsing

In [71]:
# Convert df_test into df for processing
#
df = df_sample.copy()
df.shape

(139, 33)

In [72]:
df.columns

Index(['RecordID', 'Job title', 'Full job description', 'Date collected',
       'Date posted', 'Week', 'Year', 'Month', 'Quarter', 'Hiring entity',
       'Entity size', 'Home office hybrid work', 'Full or part time status',
       'Number of applicants', 'Longitude', 'Latitude', 'City', 'Territory',
       'Region', 'Experience level', 'Education level', 'Salary start',
       'Salary end', 'Main industry', 'Source name', 'Source URL', 'File Date',
       'title', 'role', 'objectives', 'technologies', 'skills', 'summary'],
      dtype='object')

In [74]:
# Apply the parse_post_simple function to each job post
#
%%time
df['team'], df['categories'] = zip(*df['Full job description'].apply(parse_post))


CPU times: user 2.97 s, sys: 115 ms, total: 3.08 s
Wall time: 3min 1s


In [75]:
# Check that new columns are there
df.columns

Index(['RecordID', 'Job title', 'Full job description', 'Date collected',
       'Date posted', 'Week', 'Year', 'Month', 'Quarter', 'Hiring entity',
       'Entity size', 'Home office hybrid work', 'Full or part time status',
       'Number of applicants', 'Longitude', 'Latitude', 'City', 'Territory',
       'Region', 'Experience level', 'Education level', 'Salary start',
       'Salary end', 'Main industry', 'Source name', 'Source URL', 'File Date',
       'title', 'role', 'objectives', 'technologies', 'skills', 'summary',
       'team', 'categories'],
      dtype='object')

In [76]:
# Check results
#
def check_results(df, record):
    """Check the results of the parse_post_simple function."""
    print(f"team: {df.team.iloc[record]}")
    print(f"categories: {df.categories.iloc[record]}")


In [79]:
check_results(df, 20)


team: Ads Machine Learning Data Platform Team
categories: ["categories=['Machine Learning - Infrastructure']", "categories=['Advertising - Ads Quality']", "categories=['Software Engineering - Systems']"]


In [80]:
# prompt: Count the number of NaN values for select columns in df
#
nan_counts = df[['team', 'categories']].isna().sum()
print(nan_counts)


team          0
categories    0
dtype: int64


In [81]:
df.team.value_counts().nlargest(10)

,count
team,
Recommendation Algorithm Team,5
Generative AI team under Monetization Technology,4
U.S. Data Security (USDS),4
E-commerce,4
Trust and Safety,3
PDPO (Privacy and Data Protection Office),3
Applied AI (DCC) Team,3
Trust and Safety engineering team,2
Payments team of U.S. Data Security,2


In [85]:
# Write our new df to disk
#
df.to_pickle("/content/gdrive/MyDrive/Colab Notebooks/Datasets/df_ner_tiktok_5sep24.pkl")
df.to_csv("/content/gdrive/MyDrive/Colab Notebooks/Datasets/df_ner_tiktok_5sep24.csv")